# Notebook Purpose 


The purpose of this notebook is mainly to collect the training and testing data that will be used for building and evaluating the model. Data was collected using OpenCV, by accessing the webcam and then extracting the landmarks from Mediapipe. This data was then saved into the "data" array. Corresponding labels were also saved which would be used for the classification problem.

# Importing Relevant Notebooks 

In [ ]:
import copy
import itertools 
import random
import os 

import csv 
from collections import Counter 
from collections import deque

import mediapipe as mp
import cv2 
import numpy as np
import tensorflow as tf

# Steps to be implemented 

1. Access webcam with Opencv
2. Use mediapipe to process this data and extract the needed landmarks 
3. Train this data with N-Net 
4. Test the trained results
5. Pass static images to the classifier 

WEBCAM -> MEDIAPIPE LMs -> Preprocessed LMS -> MODEL Classifier -> Rock, paper scissors 

# Setting up basic utilities 

### Initializing variables for saving dataset

In [3]:
# Mediapipe hand tools 


mp_drawing = mp.solutions.drawing_utils 
mp_hands = mp.solutions.hands

NameError: name 'mp' is not defined

In [18]:
# Directory to save dataset 
save_dir = 'hand_landmarks'
os.makedirs(save_dir, exist_ok=True)

In [4]:
total_landmarks = 3000  
test_landmarks = 200 
images_per_class = total_landmarks // 3

In [25]:
landmarks = []
labels = []


In [24]:
#List to store landmarks together with labels


cap = cv2.VideoCapture(0)

# Webcam Access

### Saving data in numpy format.

In [ ]:
with mp_hands.Hands(static_image_mode= False, max_num_hands=2, min_detection_confidence=0.45) as hands:
    while True: 
        ret, frame = cap.read()
        
        if not ret:
            break
            
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False 
        results = hands.process(image)
        
        image.flags.writeable = True 
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        
        # Drawing the hand landmarks 
        
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(image, hand_landmarks
                                          , mp_hands.HAND_CONNECTIONS)
                
                image_height, image_width, _ = image.shape
                
                landmark_list = []
                
                for landmark in hand_landmarks.landmark: 
                    landmark_x = landmark.x * image_width
                    landmark_y = landmark.y * image_height
                    
                    landmark_list.extend([landmark_x, landmark_y, landmark.z])
                landmarks.append(landmark_list)
                print(landmarks)
        
        cv2.imshow('Rock Paper Scissors Classifier', image)
        
        key = cv2.waitKey(1) & 0xFF
        
        if key == ord('r') and labels.count(0) <= images_per_class:
            
            labels.append(0)
            if labels.count(0) == images_per_class:
                print("Last label for this class")
            else:
                np.save(os.path.join(save_dir, f'rock_{len(labels)}.npy'), landmarks[-1])
                print("Saved Rock Hand Landmark successfully!")
                
        elif key == ord('p') and labels.count(1) <= images_per_class:
            
            labels.append(1)
            
            if labels.count(1) == images_per_class:
                print("Last label for this class")
            else:
                np.save(os.path.join(save_dir, f'paper_{len(labels)}.npy'), landmarks[-1])
                print("Saved Paper Hand Landmark successfully!")
                
        if key == ord('s') and labels.count(2) <= images_per_class:
            
            labels.append(2)
            
            if labels.count(2) == images_per_class:
                print("Last label for this class")
            else:
                np.save(os.path.join(save_dir, f'scissors_{len(labels)}.npy'), landmarks[-1])
                print("Saved Scissors Hand Landmark successfully!")
            
        if len(labels) >= total_landmarks:
            break
        
        if key == ord('x'): 
            cap.release()
            cv2.destroyAllWindows()
            break 

Ensure you carry out the appropriate preprocessing steps for the javascript mediapipe

1. Normalizing and making a single array: Multiply the landmark x and y with the image width and heigth.


# Loading data from directory

In [26]:
labels = []

In [27]:
data = []

In [28]:
for label in range(3):
    class_dir = os.path.join(save_dir, 'class_{}'.format(label))
    print(class_dir)
    for file_name in os.listdir(class_dir):
        file_path = os.path.join(class_dir, file_name)
        landmark = np.load(file_path)
        labels.append(label)
        data.append(landmark)

hand_landmarks\class_0
hand_landmarks\class_1
hand_landmarks\class_2


# Collecting more data with Static Images for Training 

    - Extract landmarks from all the images 
    - Save the landmarks to a numpy file, and create a new directory called hand_landmarks_2 
    - Add the data to the existing data
    - Retrain the model with LSTM

In [48]:
len(IMAGE_FILES)

1000

In [71]:
IMAGE_FILES = os.listdir('rock_images')

In [72]:
IMAGE_FILES

['WIN_20230804_12_17_40_Pro (2).jpg',
 'WIN_20230804_12_17_40_Pro.jpg',
 'WIN_20230804_12_17_41_Pro (2).jpg',
 'WIN_20230804_12_17_41_Pro (3).jpg',
 'WIN_20230804_12_17_41_Pro (4).jpg',
 'WIN_20230804_12_17_41_Pro.jpg',
 'WIN_20230804_12_17_43_Pro (2).jpg',
 'WIN_20230804_12_17_43_Pro (3).jpg',
 'WIN_20230804_12_17_43_Pro.jpg',
 'WIN_20230804_12_17_44_Pro (2).jpg',
 'WIN_20230804_12_17_44_Pro (3).jpg',
 'WIN_20230804_12_17_44_Pro.jpg',
 'WIN_20230804_12_17_45_Pro (2).jpg',
 'WIN_20230804_12_17_45_Pro (3).jpg',
 'WIN_20230804_12_17_45_Pro.jpg',
 'WIN_20230804_12_17_46_Pro (2).jpg',
 'WIN_20230804_12_17_46_Pro (3).jpg',
 'WIN_20230804_12_17_46_Pro.jpg',
 'WIN_20230804_12_17_47_Pro (2).jpg',
 'WIN_20230804_12_17_47_Pro (3).jpg',
 'WIN_20230804_12_17_47_Pro (4).jpg',
 'WIN_20230804_12_17_47_Pro.jpg',
 'WIN_20230804_12_17_48_Pro (2).jpg',
 'WIN_20230804_12_17_48_Pro.jpg',
 'WIN_20230804_12_17_49_Pro (2).jpg',
 'WIN_20230804_12_17_49_Pro.jpg',
 'WIN_20230804_12_17_50_Pro (2).jpg',
 'WIN_2023

In [29]:
save_directory = '../' + save_dir

In [45]:
save_directory

'../hand_landmarks'

# Extracting Landmarks From Static Images 

### Some images were gotten from the webcam live feed, and some others were gotten from dataset online and from camera shots

In [5]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

# For static images:

with mp_hands.Hands(
    static_image_mode=True,
    max_num_hands=1,
    min_detection_confidence=0.5) as hands:
    for idx, file in enumerate(IMAGE_FILES):
    # Read an image, flip it around y-axis for correct handedness output (see
        # above).
        idx += 2002
        image = cv2.imread(file)
        #image = cv2.flip(cv2.imread(file), 1)
        # Convert the BGR image to RGB before processing.
        results = hands.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

        if not results.multi_hand_landmarks:
            print(f"No image detected - {file}")
            continue
        image_height, image_width, _ = image.shape
        annotated_image = image.copy()
        
        
        for hand_landmarks in results.multi_hand_landmarks:
            #print('hand_landmarks:', hand_landmarks)

            landmark_list = [] 
            for landmark in hand_landmarks.landmark: 
                landmark_x = landmark.x * image_width
                landmark_y = landmark.y * image_height

                landmark_list.extend([landmark_x, landmark_y, landmark.z])

            np.save(os.path.join(save_directory, f'rock_{idx}.npy'), landmark_list)
            print("Saved Rock Hand Landmark successfully!")

            mp_drawing.draw_landmarks(
              annotated_image,
              hand_landmarks,
              mp_hands.HAND_CONNECTIONS,
              mp_drawing_styles.get_default_hand_landmarks_style(),
              mp_drawing_styles.get_default_hand_connections_style())
       

NameError: name 'mp' is not defined

In [26]:
image = cv2.imread(IMAGE_FILES[0])

In [25]:
%cd rock_images

C:\Users\ssffff\Desktop\One for all\Machine Learning Work\Machine Learning projects\RPS\Model\rock_images
